In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.insert(0, '/Users/joe/pycrash')

In [3]:
os.chdir('..')

In [4]:
import pycrash
from pycrash.project import Project, project_info, load_project
from pycrash.vehicle import Vehicle
from pycrash.kinematics import SingleMotion
from pycrash.visualization.kinematics_compare import compare_kinematics
from pycrash.visualization.tire_details import tire_details, vertical_forces, long_forces
from pycrash.visualization.cg_motion_compare import cg_motion
from pycrash.visualization.model_interval import plot_motion_interval

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy import integrate
import os
pd.options.display.max_columns = None
pd.options.display.max_rows = None

Current values for defined constants:
maximum available friction (mu_max) = 0.8
time step for vehicle motion (dt) = 0.01 s


In [5]:
vehdict = {
        "year": 2015,
        "make": "Honda",
        "model": "CR-V",
        "weight": round(1531*2.2,2),
        "vin": "3CZRM3H5*FG******",
        "brake": 0,
        "steer_ratio": 15.9,
        "init_x_pos": 100,
        "init_y_pos": -12,
        "head_angle": 0,
        "width": round(182/(2.54*12),2),
        "length": round(456/(2.54*12),2),
        "hcg": round(25.94/12,2),
        "lcgf": round((0.4914*262)/(2.54*12),2),
        "lcgr": round((0.5086*262)/(2.54*12),2),
        "wb": round(262/(2.54*12),2),
        "track": round(158/(2.54*12),2),
        "f_hang": round(92/(2.54*12),2),
        "r_hang": round(102/(2.54*12),2),
        "tire_d": round(17/12,2),
        "tire_w": round(225/(25.4*12),2),
        "izz": 2199.17,
        "fwd": 1,
        "rwd": 0,
        "awd": 0,
        "A": 381,
        "B": 137,
        "k": 1000,
        "L": 0,
        "c": 0,
        "vx_initial": 0,
        "vy_initial": 0,
        "omega_z": 0
}

In [6]:
veh1 = Vehicle('veh1', vehdict)

Vehicle inputs for veh1 applied successfully


In [7]:
np.linspace(0,5,11)

array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])

In [8]:
t = np.linspace(0,5,11)
brake = [0,0,0,0,0,0,0,0,0,0,0]
throttle = [0,0,0,0,0,0.15,0.29,0.29,0.28,0.29,0]
steer = [0, 0, 0, 0, 0, 150, 250, 250, 245, 190, 185]
#steer = [0.1,0.1,0.1,0.1,0.1,150,250,250,245,190,185]
veh1.time_inputs(t,throttle,brake,steer,show_plot = True)

Driver inputs applied to veh1


In [9]:
brake = [0,0,0,0,0,0,0,0,0,0,0]
throttle = [0,0,0,0,0,0.15,0.29,0.29,0.28,0.29,0]
# steer = [0,0,0,0,0,0,0,0,0,0,0]
steer = [0,0,15,40,60,150,250,250,245,190,185]
myveh = Vehicle("Vehicle2", vehdict)
myveh.time_inputs(t,throttle,brake,steer,show_plot = False)

Vehicle inputs for Vehicle2 applied successfully
Driver inputs applied to Vehicle2


In [10]:
veh1.driver_input.to_csv('driver inputs.csv')

In [11]:
os.getcwd()

'/Users/joe/pycrash/projects/validation - single vehicle motion'

In [12]:
sim_defaults = {'dt_motion': 0.01,
                'mu_max': 0.8,
                'alpha_max': 0.174533}

In [13]:
simulation_name = 'Single Motion Test'
print(f'Creating Simulation: {simulation_name}')
run = SingleMotion(simulation_name, veh1, sim_defaults)

Creating Simulation: Single Motion Test
Maximum allowable friction: 0.8
Time step for vehicle motion (s) : 0.01
Maximum tire slip angle (deg): 10.00
Driver input for veh1 of shape = (501, 4)
Vehicle motion will be simulated for 5.0 seconds


/Users/joe/pycrash/pycrash/model_calcs/position_data.py:24: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/joe/pycrash/pycrash/model_calcs/position_data.py:60: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [14]:
plot_motion_interval(run.veh, num_itter = 5)

In [35]:
run.plot_model()

In [15]:
run.veh.model.head(10)

,t,vx,vy,Vx,Vy,Vr,oz_deg,oz_rad,delta_deg,delta_rad,turn_rX,turn_rY,turn_rR,au,av,ax,ay,ar,Ax,Ay,Ar,alphaz,alphaz_deg,beta_deg,beta_rad,lf_fx,lf_fy,rf_fx,rf_fy,rr_fx,rr_fy,lr_fx,lr_fy,lf_alpha,rf_alpha,rr_alpha,lr_alpha,lf_lock,rf_lock,rr_lock,lr_lock,lf_fz,rf_fz,rr_fz,lr_fz,theta_rad,theta_deg,Dx,Dy
0,0.000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.006289,0.00011,0.000000,0.000000,0.000000,-0.000001,0.010291,-0.000001,0.010291,0.010291,-0.000001,0.010291,0.010291,2.065625e-03,1.183516e-01,0.000000,0.000000,-0.000059,0.538212,-0.000059,0.538248,-0.000000,-0.000000,-0.000000,-0.000000,0.000110,0.000110,-0.000000,-0.000000,0.0,0.0,0.0,0.0,855.757791,855.757791,826.383953,826.383953,0.000000e+00,0.000000e+00,100.0,-12.000000
1,0.001,-1.129630e-09,0.000010,-1.129635e-09,0.000010,0.000010,0.000118,0.000002,0.006289,0.00011,4.982007,-0.000547,4.982007,-0.000001,0.010291,-0.000001,0.010291,0.010291,-0.000001,0.010291,0.010291,2.065625e-03,1.183516e-01,90.006289,1.570906,-0.000059,0.538495,-0.000059,0.537965,-0.000000,-0.000000,-0.000000,-0.000000,0.000110,0.000110,-0.000000,-0.000000,0.0,0.0,0.0,0.0,856.206677,855.308934,825.935067,826.832810,1.032813e-09,5.917581e-08,100.0,-12.000000
2,0.002,6.808056e-07,-0.012206,6.808498e-07,-0.012206,0.012206,0.000178,0.000003,0.006289,0.00011,-3939.503090,0.219739,3939.503096,0.001365,-24.443544,0.001365,-24.443544,24.443544,0.001365,-24.443544,24.443544,1.514016e-08,8.674675e-07,-89.996804,-1.570741,0.071429,-650.717071,0.071354,-650.034786,0.000000,-627.710651,-0.000000,-628.392936,-1.296379,-1.845104,-2.909600,-0.232087,0.0,0.0,0.0,0.0,856.206677,855.308934,825.935067,826.832810,3.614848e-09,2.071155e-07,100.0,-12.000006
3,0.003,-4.016798e-07,-0.012206,-4.016541e-07,-0.012206,0.012206,0.000173,0.000003,0.006289,0.00011,-4046.404528,-0.133148,4046.404530,-0.003530,24.443545,-0.003530,24.443545,24.443545,-0.003530,24.443545,24.443545,-1.637302e-04,-9.381047e-03,-90.001885,-1.570829,-0.079078,-159.933806,-0.160335,1460.658404,0.000000,1438.361551,-0.129826,-182.230641,1.570192,1.571508,1.571397,1.570084,1.0,0.0,0.0,1.0,-210.439244,1921.918964,1892.580988,-239.777219,6.672365e-09,3.822983e-07,100.0,-12.000018
4,0.004,-4.033715e-06,0.012237,-4.033853e-06,0.012237,0.012237,0.000178,0.000003,0.006289,0.00011,3930.335646,-1.295589,3930.335859,-0.003734,24.443545,-0.003734,24.443545,24.443545,-0.003734,24.443545,24.443545,3.576099e-04,2.048954e-02,90.018887,1.571126,-0.160341,1460.707292,-0.107713,-159.884941,-0.122538,-182.279535,-0.000000,1438.312703,1.570298,1.571580,1.571469,1.570190,0.0,1.0,1.0,0.0,1921.983291,-210.374970,-239.841547,1892.516715,9.737423e-09,5.579132e-07,100.0,-12.000018
5,0.005,-6.751367e-06,0.012237,-6.751465e-06,0.012237,0.012237,0.000197,0.000003,0.006289,0.00011,3556.946538,-1.962425,3556.947080,-0.001701,-24.443546,-0.001701,-24.443546,24.443546,-0.001701,-24.443546,24.443546,2.960727e-04,1.696372e-02,90.031611,1.571348,0.160341,-1460.709334,-0.157890,159.882865,-0.180405,182.281528,-0.000000,-1438.310669,-1.570358,-1.571674,-1.571786,-1.570467,0.0,1.0,1.0,0.0,1921.985977,-210.372294,-239.844233,1892.514038,1.301437e-08,7.456685e-07,100.0,-12.000006
6,0.006,-7.291943e-06,-0.012206,-7.291722e-06,-0.012206,0.012206,0.000193,0.000003,0.006289,0.00011,-3620.532073,-2.162804,3620.532719,0.000620,-24.443544,0.000620,-24.443544,24.443544,0.000620,-24.443544,24.443544,-4.339700e-04,-2.486465e-02,-90.034227,-1.571394,-0.063286,159.903254,0.160339,-1460.689068,0.000000,-1438.330992,-0.032199,182.261348,-1.570510,-1.571965,-1.572078,-1.570620,1.0,0.0,0.0,1.0,-210.399035,1921.959312,1892.540779,-239.817567,1.642027e-08,9.408119e-07,100.0,-12.000006
7,0.007,-7.773453e-06,-0.012206,-7.773272e-06,-0.012206,0.012206,0.000185,0.000003,0.006289,0.00011,-3788.938970,-2.412882,3788.939738,-0.001583,24.443544,-0.001583,24.443544,24.443544,-0.001584,24.443544,24.443544,1.342711e-04,7.693169e-03,-90.036487,-1.571433,0.016221,-159.926392,-0.160336,1460.665846,0.00000